In [5]:
import pandas as pd 

In [10]:
df_RestMex_train = pd.read_csv("Rest-Mex_2025_train.csv")

df_RestMex_train.describe()

,Polarity
count,208051.000000
mean,4.450500
std,0.932237
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [11]:
print(df_RestMex_train.columns)

Index(['Title', 'Review', 'Polarity', 'Town', 'Region', 'Type'], dtype='object')


In [ ]:
print(df_RestMex_train["Town"].value_counts())
print(f'Numero de Publos: {len(df_RestMex_train["Town"].unique().tolist())}')

Town
Tulum                         45345
Isla_Mujeres                  29826
San_Cristobal_de_las_Casas    13060
Valladolid                    11637
Bacalar                       10822
Palenque                       9512
Sayulita                       7337
Valle_de_Bravo                 5959
Teotihuacan                    5810
Loreto                         5525
TodosSantos                    4600
Patzcuaro                      4454
Taxco                          4201
Tlaquepaque                    4041
Ajijic                         3752
Tequisquiapan                  3627
Metepec                        3532
Tepoztlan                      3445
Cholula                        2790
Tequila                        2650
Orizaba                        2521
Izamal                         2041
Creel                          1786
Ixtapan_de_la_Sal              1696
Zacatlan                       1602
Huasca_de_Ocampo               1509
Mazunte                        1466
Xilitla                

In [23]:
print(df_RestMex_train["Region"].value_counts())
print(f'Numero de Regiones: {len(df_RestMex_train["Region"].unique().tolist())}')

Region
QuintanaRoo           85993
Chiapas               23532
Estado_de_Mexico      19439
Yucatan               13678
Jalisco               11168
Baja_CaliforniaSur    10125
Nayarit                7337
Puebla                 6832
Queretaro              4879
Michoacan              4454
Guerrero               4201
Morelos                3445
Veracruz               3339
San_Luis_Potosi        2218
Chihuahua              1786
Coahuila               1741
Hidalgo                1509
Oaxaca                 1466
Guanajuato              909
Name: count, dtype: int64
Numero de Regiones: 19


In [ ]:
print(df_RestMex_train["Type"].value_counts())
print(f'Numero de Tipo: {len(df_RestMex_train["Type"].unique().tolist())}')


Type
Restaurant    86720
Attractive    69921
Hotel         51410
Name: count, dtype: int64
Numero de Regiones: 3


In [35]:
conteos = df_RestMex_train.groupby(["Town", "Polarity"]).size().reset_index(name="cantidad")
tabla = conteos.pivot(index="Town", columns="Polarity", values="cantidad").fillna(0)
tabla = tabla.sort_values(by=5, ascending=False)
print(tabla)


Polarity                    1.0  2.0   3.0   4.0    5.0
Town                                                   
Tulum                       798  923  2735  9491  31398
Isla_Mujeres                649  686  1951  5756  20784
San_Cristobal_de_las_Casas  283  320  1043  2847   8567
Bacalar                     287  254   656  1722   7903
Valladolid                  285  257   851  2550   7694
Palenque                    208  247   776  2228   6053
Sayulita                    174  198   610  1712   4643
Teotihuacan                 117  100   303  1060   4230
Loreto                       83  105   325  1135   3877
Valle_de_Bravo              243  201   518  1229   3768
TodosSantos                 148  102   298   896   3156
Patzcuaro                   127  129   349   950   2899
Metepec                      88   66   216   553   2609
Ajijic                       86  109   301  1061   2195
Tlaquepaque                 251  247   447   943   2153
Tequisquiapan               182  147   376   810

In [ ]:
# Cruce Tipo vs Polarity
tabla_Type = df_RestMex_train.groupby(["Type", "Polarity"]).size().reset_index(name="cantidad")
tabla_Type_pivot = df_RestMex_train.pivot_table(index="Type", columns="Polarity", 
                                    values="Town", aggfunc="count", fill_value=0)

print("📌 Evaluaciones por Tipo y Polarity:")
print(tabla_Type_pivot)
print("\n")

# Cuántos 'Muy buenos' (Polarity=5) hay por Tipo
muy_bueno_Type = df_RestMex_train[df_RestMex_train["Polarity"] == 1]["Type"].value_counts()
print("📌 Evaluaciones 'Muy buenas' (Polarity=5) por Tipo:")
print(muy_bueno_Type)
print("\n")

📌 Evaluaciones por Tipo y Polarity:
Polarity     1.0   2.0   3.0    4.0    5.0
Type                                      
Attractive   868  1079  4895  16719  46360
Hotel       2086  2058  4622  11371  31273
Restaurant  2487  2359  6002  16944  58928


📌 Evaluaciones 'Muy buenas' (Polarity=5) por Tipo:
Type
Restaurant    2487
Hotel         2086
Attractive     868
Name: count, dtype: int64




In [ ]:

import re
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk

# Descargar recursos necesarios
nltk.download('stopwords')

# Cargar modelo de spaCy en español (puedes cambiar a 'en_core_web_sm' si es en inglés)
nlp = spacy.load("es_core_news_sm")

# --- Función para limpiar y lematizar ---
stop_words = set(stopwords.words("spanish"))

def limpiar_texto(texto):
    # Pasar a minúsculas
    texto = texto.lower()
    # Quitar caracteres no alfabéticos
    texto = re.sub(r"[^a-záéíóúñ ]", " ", texto)
    # Tokenizar y lematizar con spaCy
    doc = nlp(texto)
    tokens = [
        token.lemma_ for token in doc
        if token.is_alpha and token.lemma_ not in stop_words and len(token) > 2
    ]
    return " ".join(tokens)

# --- Aplicar limpieza a la columna Review ---
df_RestMex_train["Review_clean"] = df_RestMex_train["Review"].astype(str).apply(limpiar_texto)

# --- Generar nubes de palabras ---
def generar_nube(region, Polarity):
    subset = df_RestMex_train[(df_RestMex_train["Region"] == region) & (df_RestMex_train["Polarity"] == Polarity)]
    texto = " ".join(subset["Review_clean"])
    if len(texto.strip()) == 0:
        print(f"No hay texto para {region} - Polarity {Polarity}")
        return
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(texto)
    plt.figure(figsize=(10,5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Nube de palabras - {region} - Polarity {Polarity}")
    plt.show()

# --- Generar todas las combinaciones Region x Polarity ---
for region in df_RestMex_train["Region"].unique():
    for p in sorted(df_RestMex_train["Polarity"].unique()):
        generar_nube(region, p)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/julianrgx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
